## 추론(예측) 기반 기법과 신경망

## word2vec : 워드투벡터
2013년 구글의 토마스미콜로프(Tomas Mikolov)의 팀이 개발<br>
<b>word2vec</b> 알고리즘은 <b>신경망 모델</b>을 사용 하여 큰 텍스트 코퍼스에서 단어 연관성을 학습. 학습이 끝나면 이러한 모델은 동의어 단어를 감지하거나 부분 문장에 대한 추가 단어를 제안 할 수 있다. word2vec는 <b>벡터</b> 라고하는 특정 숫자 목록을 사용하여 각각의 고유 한 단어를 나타낸다 . 벡터는 간단한 수학적 함수 ( 벡터 간의 코사인 유사성 ) 가 해당 벡터가 나타내는 단어 간의 의미 유사성 수준을 나타내 도록 신중하게 선택 된다.



## [1] 신경망에서의 단어 처리

In [7]:
import numpy as np

text = 'You say goodbye and I say hello.'
# {0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}
# 여기서 'you'만  one-hot 인코딩으로 표현

c = np.array([[1,0,0,0,0,0,0]])
print('c:\n',c)
W = np.random.randn(7,3)
print('W:\n',W)

h = np.matmul(c,W)
print('h:\n',h)      

c:
 [[1 0 0 0 0 0 0]]
W:
 [[ 1.26558624  0.15298708 -0.79043116]
 [ 0.46565595 -0.63850571  0.14283379]
 [-1.96505793 -0.98136918 -0.54000605]
 [ 0.61348615  0.43390628 -1.29185269]
 [ 1.18122387  0.13210031  0.26647627]
 [ 0.34648755 -0.39035223  0.9697374 ]
 [ 0.08061823 -0.31632399 -1.89295021]]
h:
 [[ 1.26558624  0.15298708 -0.79043116]]


In [13]:
class Matmul:
    def __init__(self,W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.x = None
        
    def forward(self,x):
        W, = self.params
        out = np.dot(x,W)
        self.x = x
        return out
        
    def backward(self,dout):
        W, =self.params
        dx = np.dot(dout,W.T)
        dW = np.dot(self.x.T, dout)
        self.grads[0] = dw
        return dx

In [16]:
text = 'You say goodbye and I say hello.'
# {0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}
# 여기서 'you'만  one-hot 인코딩으로 표현

c = np.array([[1,0,0,0,0,0,0]])
print('c:\n',c)
W = np.random.randn(7,3)
print('W:\n',W)

layer = Matmul(W)
h = layer.forward(c)
print('h:\n',h) 

c:
 [[1 0 0 0 0 0 0]]
W:
 [[-0.06057149  0.23983351  1.99614865]
 [ 0.09029674 -0.64684241  0.56743001]
 [ 0.79363637 -2.38150638  0.44614188]
 [ 0.44452602 -0.07062423  0.34373592]
 [ 0.31022762  0.07357839  0.41361371]
 [-0.05489706 -0.08940009  1.3259452 ]
 [-0.4285702   0.39976753 -0.23331709]]
h:
 [[-0.06057149  0.23983351  1.99614865]]


## [2] 단순한 word2vec

### CBOW (Continuous Bag of Words) 모델